# Memory in Agent

This notebook goes over adding memory to an Agent. Before going through this notebook, please walkthrough the following notebooks, as this will build on top of both of them:

- [Memory in LLMChain](/docs/modules/memory/adding_memory)
- [Custom Agents](/docs/modules/agents/how_to/custom_agent)

In order to add a memory to an agent we are going to perform the following steps:

1. We are going to create an `LLMChain` with memory.
2. We are going to use that `LLMChain` to create a custom Agent.

For the purposes of this exercise, we are going to create a simple custom Agent that has access to a search tool and utilizes the `ConversationBufferMemory` class.

In [4]:
import os

from langchain.agents import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

In [5]:
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"] = "GOOGLE_CSE_ID"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

Notice the usage of the `chat_history` variable in the `PromptTemplate`, which matches up with the dynamic key name in the `ConversationBufferMemory`.

In [3]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory

prompt = hub.pull("hwchase17/react")

memory = ChatMessageHistory(session_id="test-session")

In [6]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [9]:
agent_with_chat_history.invoke(
    {"input": "How many people live in canada?"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': 'How many people live in canada?',
 'chat_history': [],
 'output': 'The current population of Canada is 39,032,811 as of Tuesday, April 9, 2024.'}

To test the memory of this agent, we can ask a followup question that relies on information in the previous exchange to be answered correctly.

In [11]:
agent_with_chat_history.invoke(
    {"input": "what is their national anthem called?"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': 'what is their national anthem called?',
 'chat_history': [HumanMessage(content='How many people live in canada?'),
  AIMessage(content='The current population of Canada is 39,032,811 as of Tuesday, April 9, 2024.')],
 'output': 'The national anthem of a country can be found by searching for the country\'s name and "national anthem".'}

We can see that the agent remembered that the previous question was about Canada, and properly asked Google Search what the name of Canada's national anthem was.

For fun, let's compare this to an agent that does NOT have memory.

In [12]:
agent = create_react_agent(llm, tools, prompt)
agent_executor_without_memory = AgentExecutor(agent=agent, tools=tools)

In [15]:
agent_executor_without_memory.invoke({"input": "How many people live in canada?"})

{'input': 'How many people live in canada?',
 'output': 'The current population of Canada is 39,032,811 as of Tuesday, April 9, 2024.'}

In [18]:
agent_executor_without_memory.invoke({"input": "what is their national anthem called?"})

{'input': 'what is their national anthem called?',
 'output': 'The national anthem of a country can be found by searching for the country\'s name and "national anthem".'}